<a href="https://colab.research.google.com/github/RMoulla/MLW_Fevrier/blob/main/Copie_de_TP_RAG_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP : Retrieval Augmented Generation (RAG)**

Dans ce TP, nous allons construire un **Retrieval Augmented Generation (RAG)**. L'objectif est de fournir une méthode complète pour traiter un fichier PDF, en extraire des informations pertinentes, puis les utiliser pour répondre à une requête utilisateur de manière augmentée. Ce TP est divisé en trois phases distinctes :

1. **Extraction et segmentation du texte** : Vous allez apprendre à extraire du texte à partir d'un fichier PDF et à le segmenter en sections de taille définie.
2. **Calcul des embeddings et recherche des segments similaires** : Nous utiliserons des techniques d'embeddings pour trouver les sections du texte les plus pertinentes par rapport à une requête donnée.
3. **Génération de réponses avec un modèle de type ChatGPT** : En utilisant le contexte extrait et le modèle GPT-4, nous allons générer une réponse contextualisée à la question posée par l'utilisateur.

Par ailleurs, nous allons utiliser des outils comme `pdfplumber`, pour parser les documents pdf, `sentence-transformers` pour les embeddings et l'API OpenAI pour générer des réponses augmentées par le contexte.

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.0 MB/s eta 0:00:00


## **Étape 1 : Extraction et segmentation du texte**

Dans cette première étape, nous allons nous concentrer sur l'extraction du texte d'un fichier PDF et sa segmentation en blocs de taille définie. L'objectif est de rendre le texte extrait plus facile à traiter dans les phases suivantes.

### Détails :
1. **Extraction du texte** : À l'aide de `pdfplumber`, vous allez extraire tout le texte d'un fichier PDF donné. Cette étape vous permettra d'obtenir une version brute du contenu du document, qui servira de base pour les autres étapes.
   
2. **Segmentation du texte** : Une fois le texte extrait, vous le diviserez en segments de taille fixe (par exemple, 500 caractères par segment) à l'aide de la fonction `textwrap.wrap()` de Python. Cela permet de découper le texte en morceaux plus faciles à manipuler dans la phase suivante où nous calculerons les similarités.

Cette étape est cruciale, car elle permet de structurer le texte pour le rendre exploitable par la suite. La qualité de l'extraction et de la segmentation influencera directement les résultats des phases suivantes.

In [ ]:
import os
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_and_tables(
    directory_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> list:
    """
    Parcourt un répertoire et renvoie une seule liste de chaînes de caractères.
    Cette liste contient, pêle-mêle :
      - Les chunks de texte issus de chaque page PDF,
      - Les chunks de texte issus de chaque table.

    Pour les tables, on les transforme en texte en
    séparant les cellules d'une ligne par "|" et
    les lignes entre elles par des sauts de ligne.
    """

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    results = []  # Une seule liste de strings

    for filename in os.listdir(directory_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    # --- 1) Texte normal ---
                    raw_text = page.extract_text() or ""
                    raw_text = raw_text.strip()

                    if raw_text:
                        chunks = text_splitter.split_text(raw_text)
                        results.extend(chunks)

                    # --- 2) Tables converties en texte ---
                    page_tables = page.extract_tables() or []
                    for table in page_tables:
                        if not table:
                            continue

                        # Construit une représentation texte de la table
                        # exemple : on sépare les cellules par " | "
                        #          et on sépare les lignes par "\n"
                        lines = []
                        for row in table:
                            # row est une liste de cellules
                            # on ignore le cas None pour éviter les erreurs de jointure
                            cleaned_row = [cell if cell else "" for cell in row]
                            line = " | ".join(cleaned_row)
                            lines.append(line)

                        table_text = "\n".join(lines).strip()

                        # Puis, on chunk ce texte comme n'importe quel texte
                        if table_text:
                            table_chunks = text_splitter.split_text(table_text)
                            results.extend(table_chunks)

    return results


In [ ]:
doc_list = extract_text_and_tables(
    directory_path="documents",
    chunk_size=500,
    chunk_overlap=200
)
doc_list[10]

'|  |  | \nQu’est-ce que l’intelligence ? |  |  | \nLa notion d’intelligence recouvre plusieurs facult´es cognitives :\n1 Raisonnement : La capacit´e `a r´esoudre des probl`emes et `a faire des\nd´eductions logiques.\n2 Apprentissage : L’aptitude `a acqu´erir de nouvelles connaissances et `a\ns’am´eliorer grˆace `a l’exp´erience.\n3 Perception : La comp´etence pour reconnaˆıtre et interpr´eter les stimuli\nsensoriels.\n4 Compr´ehension : L’habilet´e `a saisir le sens et l’importance de divers'

## **Étape 2 : Calcul des Embeddings et recherche des segments similaires**

Dans cette deuxième étape, nous allons calculer des **embeddings** pour chaque segment de texte extrait et effectuer une recherche pour identifier les segments les plus similaires à une requête utilisateur donnée.

### Détails :
1. **Calcul des embeddings** : Nous allons utiliser la bibliothèque `sentence-transformers` pour générer des vecteurs d’embeddings pour chaque segment de texte. Ces vecteurs capturent les caractéristiques sémantiques des segments, permettant ainsi de comparer leur pertinence par rapport à une requête donnée.

2. **Recherche des segments les plus similaires** : Une fois les embeddings calculés, nous allons utiliser la **similarité cosinus** pour mesurer la proximité entre l’embedding de la requête utilisateur et les embeddings des segments de texte. Les segments les plus proches seront sélectionnés pour la prochaine étape.

3. **Sélection des top-N segments** : Nous sélectionnerons les `n` segments les plus similaires à la requête, qui serviront de contexte pour générer une réponse augmentée dans la phase suivante.

Cette étape est essentielle pour filtrer le contenu extrait en fonction de sa pertinence par rapport à la requête utilisateur. Elle permet de s'assurer que seuls les segments les plus utiles sont utilisés pour générer une réponse cohérente et contextuelle.

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle pré-entraîné de sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Calcul des embeddings pour chaque segmnt de texte (tableaux NumPy par défaut)
embeddings = model.encode(doc_list)

In [ ]:

# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel est le nombre de paramètres de GPT-3 ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments


# Sélection des top-N documents les plus similaires

# Accéder aux indices des résultats top-N


# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

Segment 1 similaire : |  |  | 
Introduction aux mod`eles autor´egressifs - GPT-3 |  |  | 
GPT-3 (Generative Pre-trained Transformer 3) est le mod`ele de langue d´evelopp´e
par OpenAI, repr´esentant la troisi`eme g´en´eration de la s´erie GPT. Avec 175
milliards de param`etres, GPT-3 pousse les limites de la g´en´eration de texte et de
la compr´ehension du langage naturel.
Capacit´es g´en´erales : GPT-3 excelle dans une vari´et´e de tˆaches de TALN

Segment 2 similaire : |  |  | 
Introduction aux mod`eles autor´egressifs - GPT-3 |  |  | 
GPT-3 (Generative Pre-trained Transformer 3) est le mod`ele de langue d´evelopp´e
par OpenAI, repr´esentant la troisi`eme g´en´eration de la s´erie GPT. Avec 175
milliards de param`etres, GPT-3 pousse les limites de la g´en´eration de texte et de
la compr´ehension du langage naturel.
Capacit´es g´en´erales : GPT-3 excelle dans une vari´et´e de tˆaches de TALN

Segment 3 similaire : Architecture de GPT-3
GPT-3 repose sur une architecture Transformer am´e

## **Étape 3 : Génération de réponses avec ChatGPT**

Dans cette troisième, nous allons utiliser les segments de texte sélectionnés et une requête utilisateur pour générer une réponse contextuelle en utilisant l'API OpenAI, avec un modèle de type **ChatGPT**.

### Objectifs :
- Utiliser le modèle **GPT-4** pour générer une réponse basée sur les segments de texte les plus similaires à la requête.
- Envoyer les segments sélectionnés et la requête utilisateur sous forme de **messages** à l’API ChatGPT.
- Obtenir une réponse contextuelle, augmentée par les informations pertinentes extraites du texte.

### Détails :
1. **Création du contexte** : Nous allons concaténer les segments sélectionnés lors de l’étape précédente afin de créer un contexte cohérent à transmettre au modèle GPT-4. Ce contexte servira à fournir un maximum d’informations pertinentes pour générer une réponse précise.

2. **Appel à l’API ChatGPT** : En utilisant le contexte et la requête de l’utilisateur, nous formulerons un prompt que nous enverrons à l’API ChatGPT. Le prompt sera structuré sous forme de messages (avec les rôles "system" et "user") pour que le modèle comprenne le contexte de la conversation.

3. **Génération et récupération de la réponse** : Le modèle GPT-4 générera une réponse basée sur le contexte fourni. La réponse sera ensuite récupérée et affichée comme résultat final.

Cette étape finalise le processus de **RAG** en combinant la recherche d’informations pertinentes dans un corpus de texte et la génération de réponses intelligentes basées sur ces informations.

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.9
    Uninstalling openai-1.59.9:
      Successfully uninstalled openai-1.59.9


In [ ]:
import openai


openai.api_key = ""

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_n_paragraphs)
prompt = f"Contexte :\n{context}\n\nQuestion : {query}\nRéponse :"

# Appeler l'API d'OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4o",  # Utilisation du modèle gpt-4o
    messages=[
        {"role": "system", "content": "Tu es un assistant specialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

# Extraire et afficher la réponse générée
generated_response = response['choices'][0]['message']['content'].strip()
print(f"Réponse générée : {generated_response}")

Réponse générée : GPT-3 possède 175 milliards de paramètres.
